## ****

In [0]:
# %sql
# DROP TABLE pyspark_realtime.source.customers;

In [0]:
%sql
create table pyspark_realtime.source.customers
(
  id string,
  email string,
  city string,
  country string,
  modifiedDate timestamp
)

In [0]:
%sql
-- insert into pyspark_realtime.source.customers
-- values
-- ('1', 'john.doe@example.com', 'New York', 'USA', current_timestamp()),
-- ('2', 'jane.smith@example.com', 'London', 'UK', current_timestamp()),
-- ('3', 'mike.jones@example.com', 'Paris', 'France', current_timestamp()),
-- ('4', 'sara.williams@example.com', 'Tokyo', 'Japan', current_timestamp()),
-- ('5', 'peter.chen@example.com', 'Sydney', 'Australia', current_timestamp())




insert into pyspark_realtime.source.customers
values
('1', 'john.doe@example.com', 'Seattle', 'USA', current_timestamp()),
('6', 'jane.smith@example.com', 'London', 'UK', current_timestamp());

In [0]:
%sql
select * from pyspark_realtime.source.customers;

In [0]:
# %sql
# drop table pyspark_realtime.source.DimCustomers


In [0]:
if spark.catalog.tableExists("pyspark_realtime.source.DimCustomers"):
    pass
else:
    spark.sql("""
        CREATE TABLE pyspark_realtime.source.DimCustomers
            select *, 
                current_timestamp() as startTime,
                CAST('3000-01-01' AS TIMESTAMP) as endTime,
                'Y' as isActive
             from pyspark_realtime.source.customers
    """)

In [0]:
%sql
select * from pyspark_realtime.source.DimCustomers

## **SCD Type-2**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
df = spark.sql("""
    select * from pyspark_realtime.source.customers
""")

df = df.withColumn("dedup",row_number().over(Window.partitionBy("id").orderBy(desc("modifieddate")))).drop("dedup")
df = df.filter(col("dedup")==1)
df.createOrReplaceTempView("srctemp")

df = spark.sql("""
            select *, 
                current_timestamp() as startTime,
                CAST('3000-01-01' AS TIMESTAMP) as endTime,
                'Y' as isActive
             from srctemp
    """)

df.createOrReplaceTempView("src")


In [0]:
%sql
select * from src

## **MERGE-1: Marking the Updated Records as Expired**

In [0]:
%sql
merge into pyspark_realtime.source.dimCustomers as trg
using src as src
on trg.id=src.id and trg.isActive='Y'
when matched and src.email <> trg.email 
or src.city <> trg.city 
or src.country <> trg.country 
or src.modifiedDate <> trg.modifiedDate
then update set trg.endTime = current_timestamp(), trg.isActive = 'N'

## **MERGE-2: INSERTING NEW + UPDATED RECORDS**

In [0]:
%sql
MERGE INTO pyspark_realtime.source.DimCustomers as trg
USING src as src
ON src.id=trg.id and trg.isActive='Y'
when not matched then insert *

In [0]:
%sql    
select * from pyspark_realtime.source.DimCustomers